In [ ]:
IS_COLAB = False

In [ ]:
import sys
import os
import gc
import torch
import pandas as pd
import numpy as np
import glob
import math
import pickle

import matplotlib.pyplot as plt
from torch.utils.data.dataset import random_split
from torch.utils.data import Dataset, DataLoader

import torch.nn as nn
import torch.nn.functional as F
import torch

from sklearn import metrics

# Log progress
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

if IS_COLAB:
    # using google colab
    from google.colab import drive
    drive.mount('/content/drive')
    ROOT = os.path.join('/content', 'drive', 'My Drive',
                        'concurrent-activity-recognition')
    os.chdir(ROOT)
else:
    # using AWS
    ROOT = os.path.join('/home', 'ubuntu', 'concurrent-activity-recognition')
    os.chdir(ROOT)

print(ROOT)

if torch.cuda.is_available():
#if False:
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

In [ ]:
# dataset constant
FEATURE_LENGTH = 36
LABEL_LENGTH = 3
FILE_LENGTH = 30
DATASET_DIR = "dataset_parsed_all_features_label0_label1"
LABEL_INDEX = 0
LABEL_COUNT = 3

class OpportunityDatasetParsed(Dataset):
    def __init__(self, filenames):
        self.filenames = filenames
        
    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, index):
        filename = self.filenames[index]
        #print(filename)

        df = pd.read_csv(filename, header=0)

        # first FEATURE_LENGTH columns are for data
        data = torch.FloatTensor(
            df.iloc[:, :FEATURE_LENGTH].values.astype('float'))
                
        # last LABEL_LENGTH columns are for label
        label = torch.FloatTensor(
            df.iloc[FILE_LENGTH-1, FEATURE_LENGTH:(FEATURE_LENGTH+LABEL_LENGTH)].values.astype('float'))

        return data, label

In [ ]:
USE_MODEL = "CNNLSTM"
MODEL_NAME = "CNNLSTM_label0_label1"

# grid search for CNNLSTM
if USE_MODEL == "CNNLSTM":
    conv1_outchs = [20]
    conv1_sizes = [3]
    conv2_outchs = [40]
    conv2_sizes = [3]
    lstm1_hs = [20]
    learning_rates = [0.01]

    grids = [(conv1_outch, conv1_size, conv2_outch,
            conv2_size, lstm1_h, learning_rate)
            for conv1_outch in conv1_outchs
            for conv1_size in conv1_sizes
            for conv2_outch in conv2_outchs
            for conv2_size in conv2_sizes
            for lstm1_h in lstm1_hs
            for learning_rate in learning_rates
            ]

if USE_MODEL == "CNN_Custom":
    grids = [1]

print("Total grid parameters: " + str(len(grids)))

In [ ]:
# training hyperparameter
num_epochs = 50
batch_size = 8

class CNN_LSTM(nn.Module):
    def __init__(self, grid):
        super(CNN_LSTM, self).__init__()
        conv1_outch = grid[0]
        conv1_size = grid[1]
        conv1_outlen = FILE_LENGTH - (conv1_size - 1)
        conv2_outch = grid[2]
        conv2_size = grid[3]
        conv2_outlen = conv1_outlen - (conv2_size - 1)
        lstm1_h = grid[4]
        self.lstm1_h = lstm1_h
        lstm1_hlen = conv2_outlen
        self.lstm1_hlen = lstm1_hlen
        
        # 2nd LSTM layer
        self.lstm2_h = lstm1_h
        lstm2_hlen = lstm1_hlen
        self.lstm2_hlen = lstm2_hlen

        self.conv1 = nn.Conv1d(FEATURE_LENGTH, conv1_outch, conv1_size)
        self.conv2 = nn.Conv1d(conv1_outch, conv2_outch, conv2_size)
        self.lstm1 = nn.LSTM(conv2_outch, lstm1_h)
        self.fc1 = nn.Linear(lstm1_h*lstm1_hlen, 3)
        
        # 2nd LSTM layer
        self.lstm2 = nn.LSTM(lstm1_h, self.lstm2_h)
        self.fc2 = nn.Linear(lstm1_h*lstm1_hlen, 13)

    def forward(self, x):
        h = self.init_hidden()
        batch_len = x.shape[0]
        x.transpose_(1, 2)
        x = F.relu(self.conv1(x), inplace=False)
        x = F.relu(self.conv2(x), inplace=False)
        x.transpose_(1, 2)
        x1ini, _ = self.lstm1(x, h)
        x1 = x1ini.reshape([batch_len, -1])
        out1 = self.fc1(x1)
        
#         # first layer
#         return out1
        
        # second layer
        x2, _ = self.lstm2(x1ini, h)
        x2 = x2.reshape([batch_len, -1])
        out2 = self.fc2(x2)
        return out1, out2

    def init_hidden(self):
        weight = next(self.parameters()).data
        hidden = (weight.new(1, self.lstm1_hlen,
                             self.lstm1_h).zero_().to(device),
                  weight.new(1, self.lstm1_hlen,
                             self.lstm1_h).zero_().to(device))
        return hidden

class CNN_Custom(nn.Module):
    def __init__(self):
        super(CNN_Custom, self).__init__()
        self.conv1 = nn.Conv2d(1, 50, (1, 5))
        self.pool1 = nn.MaxPool2d((1, 2))
        self.conv2 = nn.Conv2d(50, 40, (1, 5))
        self.pool2 = nn.MaxPool2d((1, 3))
        self.conv3 = nn.Conv2d(40, 20, (1, 3))
        self.fc1 = nn.Linear(FEATURE_LENGTH*20, LABEL_COUNT)

        self.conv4 = nn.Conv1d(20, 20, FEATURE_LENGTH, groups=20)
        self.fc2 = nn.Linear(20, LABEL_COUNT)

    def forward(self, x):
        batch_len = x.shape[0]
        x = x.reshape(x.shape[0], 1, x.shape[1], x.shape[2])
        x.transpose_(2, 3)
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = F.relu(self.conv3(x))
        #print(x.shape)

        # simple concatenation. It works better in the end
        x = x.reshape([batch_len, -1])
        x = self.fc1(x)

#         # depth-wise convolution for parameter concatenation
#         x = x.reshape([batch_len, 20, 36])
#         x = F.relu(self.conv4(x))
#         x = x.reshape([batch_len, -1])
#         x = self.fc2(x)
        return x

def make_train_step(model, loss_function, optimizer):
    def train_step(data, labels):
        model.train()
        optimizer.zero_grad()

        data, labels = data.to(device), labels.to(device)

        data = torch.reshape(data, (len(labels), -1, FEATURE_LENGTH))

        label1 = torch.reshape(labels[:, LABEL_INDEX:LABEL_INDEX+1].long(), (-1,))
        
#         # for 1 label
#         output1 = model(data)
#         loss = loss_function(output1, label1)
#         loss.backward()
#         optimizer.step()
        
        # for 2 labels
        output1, output2 = model(data)
        
        label2 = torch.reshape(labels[:, 1:1+1].long(), (-1,))
        
        loss = loss_function(output1, label1)
        loss.backward(retain_graph=True)
        optimizer.step()
        
        loss = loss_function(output2, label2)
        loss.backward()
        optimizer.step()
        
        return loss.item()

    return train_step

In [ ]:
# Training and validation, by using grid search/configured parameters
LEARNING_RATE = 0.01
TRAIN_RATIO = 0.8

torch.manual_seed(69)
models = []
fig, axs = plt.subplots(1, figsize=(18, 4*1))
i = 0
for grid in grids:
    if USE_MODEL == "CNNLSTM":
        model = CNN_LSTM(grid).to(device)
    elif USE_MODEL == "CNN_Custom":
        model = CNN_Custom().to(device)
    loss_function = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)
    train_step = make_train_step(model, loss_function, optimizer)

    data_train_files = glob.glob(DATASET_DIR + "/train/*.csv")
    train_losses = []
    val_losses = []

    data_train = OpportunityDatasetParsed(data_train_files)
    train_dataset, val_dataset = random_split(data_train,
            [math.ceil(TRAIN_RATIO*len(data_train)), math.floor((1 - TRAIN_RATIO)*len(data_train))])

    for epoch in log_progress(range(num_epochs), name="Epoch"):
        dataloader_train = DataLoader(train_dataset, batch_size=batch_size,
                        shuffle=True)
        
        train_losses_temp = []
        for data, labels in dataloader_train:
            loss = train_step(data, labels)
            train_losses_temp.append(loss)

        train_losses.append(np.mean(train_losses_temp))

        with torch.no_grad():
            dataloader_val = DataLoader(val_dataset, batch_size=batch_size,
                            shuffle=False)
            
            val_losses_temp = []
            for data, labels in dataloader_val:
                model.eval()
                data, labels = data.to(device), labels.to(device)

                label1 = torch.reshape(labels[:, LABEL_INDEX:LABEL_INDEX+1].long(), (-1,))
                
#                 # for 1 label
#                 output1 = model(data)
#                 val_loss = loss_function(output1, label1)
#                 val_losses_temp.append(val_loss.item())
                
                # for 2 labels
                output1, output2 = model(data)
                label2 = torch.reshape(labels[:, 1:1+1].long(), (-1,))
                val_loss1 = loss_function(output1, label1)
                val_loss2 = loss_function(output2, label2)
                val_loss_avg = (val_loss1.item() + val_loss2.item())/2
                val_losses_temp.append(val_loss2.item())

                
            
            val_losses.append(np.mean(val_losses_temp))

    models.append(model)

    try:
        axs[i].plot(range(0, num_epochs), train_losses, label="train_loss")
        axs[i].plot(range(0, num_epochs), val_losses, label="val_loss")
        axs[i].set_title(grid)
    except:
        axs.plot(range(0, num_epochs), train_losses, label="train_loss")
        axs.plot(range(0, num_epochs), val_losses, label="val_loss")
        axs.set_title(grid)
    i += 1

plt.legend()

In [ ]:
# test and save the model
from datetime import datetime
import pickle
data_test_files = glob.glob(DATASET_DIR + "/test/*.csv")
data_test = OpportunityDatasetParsed(data_test_files)

i = 0
for model in models:
    with torch.no_grad():
        #h = model.init_hidden()
        dataloader_test = DataLoader(data_test, batch_size=batch_size,
                        shuffle=False)
        
        label1_all = []
        predicted1_all = []

        for data, labels in log_progress(dataloader_test, name="Testing"):
            model.eval()
            data, labels = data.to(device), labels.to(device)

            label1 = torch.reshape(labels[:, LABEL_INDEX:LABEL_INDEX+1].long(), (-1,))
            label1_all.extend(label1.tolist())

            output1 = model(data)

            _, predicted1 = torch.max(output1, 1)

            predicted1_all.extend(predicted1.tolist())
        
        print("Grid: {}".format(grids[i]))
        print("Accuracy: {}".format(metrics.accuracy_score(label1_all, predicted1_all)))
        print("F1 (micro): {}".format(metrics.f1_score(label1_all, predicted1_all, average='micro')))
        print("F1 (macro): {}".format(metrics.f1_score(label1_all, predicted1_all, average='macro')))
        print("F1 (weighted): {}".format(metrics.f1_score(label1_all, predicted1_all, average='weighted')))

    with open("models/" + MODEL_NAME + "_" + datetime.now().strftime("%d%m%Y%H%M%S") + ".p", 'wb') as file:
        pickle.dump(model, file)
    
    i += 1

In [ ]:
# load model and test it, just to be sure
loaded_model = pickle.load(open("models/CNN_Custom_label1_19112020084756.p", 'rb'))

data_test_files = glob.glob("dataset_parsed_all_features_label1" + "/test/*.csv")
data_test = OpportunityDatasetParsed(data_test_files)

with torch.no_grad():
    dataloader_test = DataLoader(data_test, batch_size=batch_size,
                    shuffle=False)

    label1_all = []
    predicted1_all = []

    for data, labels in log_progress(dataloader_test, name="Testing"):
        loaded_model.eval()
        data, labels = data.to(device), labels.to(device)

        label1 = torch.reshape(labels[:, LABEL_INDEX:LABEL_INDEX+1].long(), (-1,))
        label1_all.extend(label1.tolist())

        output1 = loaded_model(data)

        _, predicted1 = torch.max(output1, 1)

        predicted1_all.extend(predicted1.tolist())

    print("Accuracy: {}".format(metrics.accuracy_score(label1_all, predicted1_all)))
    print("F1 (micro): {}".format(metrics.f1_score(label1_all, predicted1_all, average='micro')))
    print("F1 (macro): {}".format(metrics.f1_score(label1_all, predicted1_all, average='macro')))
    print("F1 (weighted): {}".format(metrics.f1_score(label1_all, predicted1_all, average='weighted')))

In [ ]:
predicted1_all

In [ ]:
# combine two models for prediction of label0 and label1
import pickle
loaded_model_label0 = pickle.load(open("models/CNN_Custom_label0_19112020090508.p", 'rb'))
loaded_model_label1 = pickle.load(open("models/CNN_Custom_label1_19112020090833.p", 'rb'))

data_test_files = glob.glob("dataset_parsed_few_features_label0_label1" + "/test/*.csv")
data_test = OpportunityDatasetParsed(data_test_files)

with torch.no_grad():
    dataloader_test = DataLoader(data_test, batch_size=batch_size,
                    shuffle=False)

    label0_all = []
    predicted0_all = []
    label1_all = []
    predicted1_all = []

    for data, labels in log_progress(dataloader_test, name="Testing"):
        loaded_model_label0.eval()
        loaded_model_label1.eval()
        data, labels = data.to(device), labels.to(device)

        label0 = torch.reshape(labels[:, 0:0+1].long(), (-1,))
        label0_all.extend(label0.tolist())
        
        label1 = torch.reshape(labels[:, 1:1+1].long(), (-1,))
        label1_all.extend(label1.tolist())

        output0 = loaded_model_label0(data)
        output1 = loaded_model_label1(data)

        _, predicted0 = torch.max(output0, 1)
        _, predicted1 = torch.max(output1, 1)

        predicted0_all.extend(predicted0.tolist())
        predicted1_all.extend(predicted1.tolist())

    print("Label0:")
    print("Accuracy: {}".format(metrics.accuracy_score(label0_all, predicted0_all)))
    print("F1 (micro): {}".format(metrics.f1_score(label0_all, predicted0_all, average='micro')))
    print("F1 (macro): {}".format(metrics.f1_score(label0_all, predicted0_all, average='macro')))
    print("F1 (weighted): {}".format(metrics.f1_score(label0_all, predicted0_all, average='weighted')))
    
    print("Label1:")
    print("Accuracy: {}".format(metrics.accuracy_score(label1_all, predicted1_all)))
    print("F1 (micro): {}".format(metrics.f1_score(label1_all, predicted1_all, average='micro')))
    print("F1 (macro): {}".format(metrics.f1_score(label1_all, predicted1_all, average='macro')))
    print("F1 (weighted): {}".format(metrics.f1_score(label1_all, predicted1_all, average='weighted')))

In [ ]:
# for LSTM 2 labels
data_test_files = glob.glob(DATASET_DIR + "/test/*.csv")
data_test = OpportunityDatasetParsed(data_test_files)

with torch.no_grad():
    dataloader_test = DataLoader(data_test, batch_size=batch_size,
                    shuffle=False)

    label0_all = []
    predicted0_all = []
    label1_all = []
    predicted1_all = []

    for data, labels in log_progress(dataloader_test, name="Testing"):
        model.eval()
        data, labels = data.to(device), labels.to(device)

        label0 = torch.reshape(labels[:, 0:0+1].long(), (-1,))
        label0_all.extend(label0.tolist())
        
        label1 = torch.reshape(labels[:, 1:1+1].long(), (-1,))
        label1_all.extend(label1.tolist())

        output0, output1 = model(data)

        _, predicted0 = torch.max(output0, 1)
        _, predicted1 = torch.max(output1, 1)

        predicted0_all.extend(predicted0.tolist())
        predicted1_all.extend(predicted1.tolist())

    print("Label0:")
    print("Accuracy: {}".format(metrics.accuracy_score(label0_all, predicted0_all)))
    print("F1 (micro): {}".format(metrics.f1_score(label0_all, predicted0_all, average='micro')))
    print("F1 (macro): {}".format(metrics.f1_score(label0_all, predicted0_all, average='macro')))
    print("F1 (weighted): {}".format(metrics.f1_score(label0_all, predicted0_all, average='weighted')))
    
    print("Label1:")
    print("Accuracy: {}".format(metrics.accuracy_score(label1_all, predicted1_all)))
    print("F1 (micro): {}".format(metrics.f1_score(label1_all, predicted1_all, average='micro')))
    print("F1 (macro): {}".format(metrics.f1_score(label1_all, predicted1_all, average='macro')))
    print("F1 (weighted): {}".format(metrics.f1_score(label1_all, predicted1_all, average='weighted')))

In [ ]:
data_train_files = glob.glob(DATASET_DIR + "/train/*.csv")
data_test_files = glob.glob(DATASET_DIR + "/test/*.csv")

In [ ]:
len(data_train_files)

In [ ]:
len(data_test_files)

In [ ]:
predicted1_all